In [1]:
import pandas as pd

import numpy as np

from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate, GridSearchCV

import joblib

import os

In [5]:
# Tìm đường dẫn thư mục con và thư mục chính
current_dir = os.path.abspath(os.getcwd())
parrent_dir = os.path.dirname(current_dir)
base_dir = os.path.dirname(parrent_dir)

In [8]:
cols_to_use_users = ['reviewerID', 'asin', 'vaderSentimentScores']  # Ví dụ cột cần thiết cho người dùng
df = pd.read_csv(base_dir + "\\Dữ liệu đã được xử lý\\largerReviewDataset.csv", usecols=cols_to_use_users)

In [9]:
df

,reviewerID,asin,vaderSentimentScores
0,AXHY24HWOF184,0470536454,5
1,A29OWR79AM796H,0470536454,4
2,AUPWU27A7X5F6,0470536454,4
3,A1N69A47D4JO6K,0470536454,3
4,AHTIQUMVCGBFJ,0470536454,5
...,...,...,...
1509235,A311NUWMD55WPO,B01HJETSF2,4
1509236,AXJFL8T55OXJG,B01HJETSF2,5
1509237,A1RHBI07JTQBER,B01HJETSF2,5
1509238,A1V0G9QMGY1RGK,B01HJETSF2,5


In [10]:
ratings_df = df.copy()

In [11]:
# Bước 1: Giảm kích thước dữ liệu (nếu cần)
# Lọc dữ liệu để giữ lại chỉ những người dùng và items có ít nhất X đánh giá
min_ratings = 10
filtered_ratings = ratings_df.groupby('reviewerID').filter(lambda x: len(x) >= min_ratings)
filtered_ratings = filtered_ratings.groupby('asin').filter(lambda x: len(x) >= min_ratings)
filtered_ratings

,reviewerID,asin,vaderSentimentScores
1185,A3CEV9KBYLPK2L,1480360295,2
1209,AUKVM8TXMBUR5,1480360295,5
1258,A1IVXL7131OOIW,1480360295,5
1270,A2053ZJUGCKUA5,1480360295,3
1310,A7P4VCO9K63ZD,1480360295,3
...,...,...,...
1509142,AWSRO7PPP06P6,B01HI1VM28,5
1509145,A1T0W8JXB4USYB,B01HI1VM28,2
1509147,A2CCA7U66RSXD5,B01HI1VM28,5
1509148,A2NYLAJ4PI0CVK,B01HI1VM28,4


In [12]:
print("Total No of reviewerID: ", len(np.unique(filtered_ratings.reviewerID)))
print("Total No of asin: ", len(np.unique(filtered_ratings.asin)))

Total No of reviewerID:  9953
Total No of asin:  3020


In [13]:
# Bước 2: Tạo tập dữ liệu Surprise
reader = Reader(rating_scale=(1, 5))  # Thay đổi rating_scale nếu cần
data = Dataset.load_from_df(filtered_ratings[['reviewerID', 'asin', 'vaderSentimentScores']], reader)

In [14]:
# Bước 3: Tối ưu hóa mô hình SVD
param_grid = {
    'n_epochs': [5, 10, 20],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.02, 0.1, 0.2]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs=-1)
gs.fit(data)
# In ra các tham số tốt nhất và hiệu suất tương ứng
print(f"Best RMSE: {gs.best_score['rmse']}, Best parameters: {gs.best_params['rmse']}")

Best RMSE: 0.8990401470996353, Best parameters: {'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.02}


In [15]:
# Bước 4: Huấn luyện mô hình với các tham số tốt nhất và đánh giá
best_model = SVD(**gs.best_params['rmse'])
trainset = data.build_full_trainset()
best_model.fit(trainset)

In [16]:
cross_validate(best_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8899  0.8793  0.8874  0.8928  0.8815  0.8862  0.0051  
MAE (testset)     0.6144  0.6063  0.6127  0.6139  0.6101  0.6115  0.0030  
Fit time          0.72    0.72    0.75    0.79    0.77    0.75    0.03    
Test time         0.15    0.07    0.07    0.08    0.15    0.10    0.04    


{'test_rmse': array([0.88992285, 0.8792758 , 0.88739229, 0.89277882, 0.88149308]),
 'test_mae': array([0.61440944, 0.6062852 , 0.61274694, 0.61385656, 0.61011789]),
 'fit_time': (0.7171480655670166,
  0.722503662109375,
  0.7451677322387695,
  0.7931885719299316,
  0.7658717632293701),
 'test_time': (0.14796733856201172,
  0.06943702697753906,
  0.07179474830627441,
  0.07662725448608398,
  0.14703011512756348)}

In [17]:
# Lưu mô hình SVD với joblib
model_file = parrent_dir + "\\Best recommendation models\\SVD_CF.pkl"  
joblib.dump(best_model, model_file)

['d:\\Source_code\\3. Hệ thống đề xuất\\Best recommendation models\\SVD_CF.pkl']

In [18]:
# Hàm để tạo gợi ý
def get_collaborative_filtering_recommendations(user_id, top_n=10):
    # Tạo testset cho người dùng cụ thể
    testset = trainset.build_anti_testset()
    testset = list(filter(lambda x: x[0] == user_id, testset))
    predictions = best_model.test(testset)
    predictions.sort(key=lambda x: x.est, reverse=True)
    return [pred.iid for pred in predictions[:top_n]]

In [19]:
# Lấy gợi ý cho người dùng
user_id = "A3CEV9KBYLPK2L"
top_n = 10
recommendations = get_collaborative_filtering_recommendations(user_id, top_n)
print(recommendations)

['B01GUTT7JE', 'B002TUTUIM', 'B000YE8L7G', 'B004VQDAQK', 'B001L8PIPW', 'B00FPT8D4Q', 'B009AF7TBY', 'B0016OMSYU', 'B0073XH0WS', 'B00BKVHIUI']
